# Validating ADES Observation Files

#### This tutorial demonstrates how to use the `iau-ades` Python package to validate ADES-formatted observation files locally, before submitting them to the Minor Planet Center.

The [ADES (Astrometry Data Exchange Standard)](https://www.minorplanetcenter.net/mpcops/documentation/ades/) is the standard format for submitting astrometric observations to the MPC. Observations can be formatted as either XML or PSV (Pipe-Separated Values).

The [`iau-ades`](https://github.com/IAU-ADES/ADES-Master) Python package provides tools to:
 - **Validate** ADES XML files against the submission schema (`submit.xsd`)
 - **Convert** between PSV and XML formats

Validating your ADES files locally before submission is a useful step that can help identify formatting errors early, saving time in the submission process.

Further information and documentation on the ADES standard can be found at:
 - https://www.minorplanetcenter.net/mpcops/documentation/ades/
 - https://github.com/IAU-ADES/ADES-Master
 - https://www.minorplanetcenter.net/mpcops/documentation/valid-ades-values/

# Install & Import Packages

Here we install the `iau-ades` package and import the standard Python packages we use in this tutorial.

In [ ]:
!pip install -q iau-ades requests

In [ ]:
import requests
import tempfile
import atexit
import os
from ades.valsubmit import valsubmit
from ades.psvtoxml import psvtoxml

# Helper Function

The `valsubmit` function writes its results to a file called `valsubmit.file` in the current working directory. Here we define a small helper function that calls `valsubmit` and then reads and displays the result.

In [ ]:
def validate_and_report(xml_filepath):
    """
    Validate an ADES XML file and print the result.
    
    Parameters
    ----------
    xml_filepath : str
        Path to the XML file to validate.
    """
    valsubmit(xml_filepath)
    with open('valsubmit.file') as f:
        result = f.read().strip()
    print(f"Validation result: {result}")

# Sample ADES Files

Here we download sample XML and PSV files from the MPC's website and save them to local (temporary) files.

These files are known to be valid submissions to the MPC.

If you have created your own ADES file(s), then this download of sample files is not necessary.

The local, temporary files will be deleted after this notebook exits.

**Note:** The sample files from the MPC use ADES version `2017`. The current `iau-ades` package validates against the version `2022` schema. Below, we update the version and fix any fields that have changed between versions.

In [ ]:
# Define the files to fetch
mpc_urls = [
    "https://data.minorplanetcenter.net/media/ades/goodsubmit.xml.txt",
    "https://data.minorplanetcenter.net/media/ades/goodsubmit.psv.txt"
]

local_filenames = ["sample.xml", "sample.psv"]
temp_filepaths = {}

for local_filename, mpc_url in zip(local_filenames, mpc_urls):
    content = requests.get(mpc_url).text

    # Update from version 2017 to 2022
    content = content.replace('version="2017"', 'version="2022"').replace('version=2017', 'version=2022')
    # In v2022, RA values must not have a leading '+' sign
    content = content.replace('<ra>+', '<ra>').replace('<dec>+', '<dec>')
    content = content.replace('| +0.1      |+30.0', '|  0.1      | 30.0')

    ext = local_filename[-3:]  # 'xml' or 'psv'
    with tempfile.NamedTemporaryFile(mode='w', suffix='.' + ext, delete=False) as f:
        f.write(content)
        temp_filepaths[ext] = f.name
        print(f"Saved {ext.upper()} sample to: {temp_filepaths[ext]}")
        atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

# Validate XML Format Data

The `iau-ades` package validates ADES files in **XML format**. The `valsubmit` function checks an XML file against the ADES submission schema (`submit.xsd`).

### XML Validation: Successful

Here we validate the sample XML file downloaded above. Since this file contains correctly formatted ADES data, the validation should pass.

In [ ]:
validate_and_report(temp_filepaths['xml'])

### XML Validation: Failed (RA out of range)

Here we create an intentionally invalid XML file where the Right Ascension (`ra`) value is set to `999.999`, which exceeds the allowed maximum of `360.0` degrees. The validation should fail and report the error.

In [ ]:
# Read the good XML and introduce an error: set RA to an invalid value
with open(temp_filepaths['xml']) as f:
    good_xml = f.read()

bad_xml = good_xml.replace('<ra>0</ra>', '<ra>999.999</ra>', 1)

# Write the bad XML to a temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as f:
    f.write(bad_xml)
    bad_xml_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

# Validate the bad XML
validate_and_report(bad_xml_path)

### XML Validation: Failed (missing required element)

Here we create another invalid XML file where the required `stn` (station/observatory code) element has been removed from an observation. The validation should fail because `stn` is a required field.

In [ ]:
# Remove the <stn>F51</stn> element from the first observation
bad_xml_nostn = good_xml.replace('        <stn>F51</stn>\n', '', 1)

# Write the bad XML to a temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as f:
    f.write(bad_xml_nostn)
    bad_xml_nostn_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

# Validate the bad XML
validate_and_report(bad_xml_nostn_path)

# Validate PSV Format Data

The `iau-ades` validation functions operate on **XML** format. If your observations are in PSV (Pipe-Separated Values) format, they must first be converted to XML before validation can be performed.

The `iau-ades` package provides the `psvtoxml` function for this conversion. This mirrors what the MPC does internally when PSV files are submitted.

### PSV Validation: Successful

Here we demonstrate the two-step process:
1. Convert the PSV file to XML using `psvtoxml`
2. Validate the resulting XML using `valsubmit`

In [ ]:
# Step 1: Convert PSV to XML
with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as f:
    xml_from_psv_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

psvtoxml(temp_filepaths['psv'], xml_from_psv_path)
print(f"Converted PSV to XML: {xml_from_psv_path}")

# Step 2: Validate the converted XML
validate_and_report(xml_from_psv_path)

### PSV Validation: Failed

Here we create an invalid PSV file where one observation has a Declination (`dec`) value of `91` (exceeding the allowed maximum of `90.0` degrees). We convert it to XML and then validate to show that the error is caught.

**Note:** The `psvtoxml` function uses module-level state that is not reset between calls. We use `importlib.reload` to reset this state before the second conversion.

In [ ]:
# Reset psvtoxml module state (required when calling psvtoxml multiple times)
import importlib
import ades.psvtoxml
importlib.reload(ades.psvtoxml)
from ades.psvtoxml import psvtoxml

# Read the good PSV and introduce an error: set Dec to an invalid value
with open(temp_filepaths['psv']) as f:
    good_psv = f.read()

bad_psv = good_psv.replace('| 90        |', '| 91        |', 1)

# Write the bad PSV to a temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.psv', delete=False) as f:
    f.write(bad_psv)
    bad_psv_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

# Step 1: Convert bad PSV to XML
with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as f:
    bad_xml_from_psv_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

psvtoxml(bad_psv_path, bad_xml_from_psv_path)
print("PSV converted to XML successfully (conversion does not validate content).")

# Step 2: Validate the converted XML
validate_and_report(bad_xml_from_psv_path)

# Summary

In this tutorial we demonstrated how to use the `iau-ades` Python package to validate ADES-formatted observation files locally:

 - **XML files** can be validated directly using `valsubmit` from `ades.valsubmit`
 - **PSV files** must first be converted to XML using `psvtoxml` from `ades.psvtoxml`, and then validated
 - Validation checks the data against the ADES submission schema (`submit.xsd`) and reports errors such as out-of-range values or missing required elements

We recommend validating your ADES files locally before submitting them to the MPC. For information on how to submit validated files to the MPC, see the [Submit Observations](mpc_tutorial_api_submission_submission.ipynb) tutorial.